# BERT

Transformer的Encoder就是BERT架构，BERTtrain这个encoder只需要投入很多预料就行，**而不需要用label**，输入一个句子，输出一堆embedding

用中文的character的实操性更好，因为常用字只有4000个，最好不用词

## 1 BERT原理

![](./pics/BERT/1.png)

### 1.1 BERT训练

有两种方式训练，这两个方法要同时使用，BERT会学的很好

训练方式一：挖空法 Mask LN

输入的句子有15%的概率会被置换成token，被盖掉，BERT训练的时候去猜测里面是什么字   
通过BERT输出一堆embedding，然后将挖空的部分丢进一个线性的分类器，问分类器挖掉的词汇是哪一个？  
由于这是一个线性的分类器，所以能预测对的能力很弱，所以需要BERT得到的embedding的representation很好  

![](./pics/BERT/masked-ln.png)

训练方式二：下一句预测 Next Sentence Prediction

给BERT两个句子，让BERT去预测这两个句子是不是应该接在一起  
这里需要引入一个特殊的符号就是[SEP]，告诉两个句子的交接在哪里  
还要给一个[CLS]告诉在这里要做分类，这个[CLS]输出的embedding再丢掉一个简单的线性二分类器，来预测是不是应该接在一起  
由于BERT是self-attention，所以天涯若比邻，[CLS]放在开头和结尾其实没有差别

![](./pics/BERT/next-sentence-prediction.png)

### 1.2 BERT使用

#### 1.2.1 输入句子输出类别

输入一个句子，来告诉是正面还是负面的，或者文章的分类  
把这个句子丢给BERT，然后句子开头加上类别[CLS]，输出结果丢给一个线性分类器预测类别  
BERT的参数和线性分类器可以一起学，线性分类器从头学，BERT微调fine-tune就可以

![](./pics/BERT/case1.png)

#### 1.2.2 输入句子输出句子每个词汇的类别

比如slot filling，输入一个句子，输出每个词汇属于哪个slot  
输入一个句子，输出一堆embdedding，然后看属于哪个class  
也是端到端的训练，将线性分类器和BERT一起训练

![](./pics/BERT/case2.png)

#### 1.2.3 输入两个句子输出类别

比如机器推论，给机器一个前提，一个假设，来推断假设是错还是对

![](./pics/BERT/case3.png)

#### 1.2.4 Extraction-based Q&A

文章的答案都在文章中  
答案是文章中的位置，ds到de这几个token  

![](./pics/BERT/case4_1.png)

如果s=3，e=2，所以就没有答案   
红色vector和每个黄色的词汇embedding做点积后，然后进行softmax，看哪个词汇得到的分数最高，红色的决定了s，蓝色的决定了e  
红色的vector和蓝色的vector是学习出来的，BERT fine-tune就好，需要给很多问题、文章和答案落在第几个词汇

![](./pics/BERT/case4_2.png)